In [1]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [2]:
import numpy as np
import pandas as pd
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [3]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

In [4]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [5]:
# reflect an existing database into a new model
base = automap_base()

# reflect the tables
base.prepare(engine, reflect = True)

In [6]:
# View all of the classes that automap found
base.classes.keys()

['measurement', 'station']

In [7]:
# Save references to each table
Measurement = base.classes.measurement
Station = base.classes.station

In [8]:
# Create our session (link) from Python to the DB
session = Session(engine)


In [9]:
# Display the Measurement row's columns and data in dictionary format
first_row = session.query(Measurement).first()
first_row.__dict__

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x1e063288308>,
 'id': 1,
 'station': 'USC00519397',
 'prcp': 0.08,
 'date': '2010-01-01',
 'tobs': 65.0}

In [10]:
# Display the Station row's columns and data in dictionary format
first_row = session.query(Station).first()
first_row.__dict__

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x1e06328fe08>,
 'name': 'WAIKIKI 717.2, HI US',
 'longitude': -157.8168,
 'latitude': 21.2716,
 'station': 'USC00519397',
 'id': 1,
 'elevation': 3.0}

# Exploratory Precipitation Analysis

In [11]:
# Find the most recent date in the data set.
most_recent = session.query(Measurement).order_by(Measurement.date.desc()).first()
newest_date = most_recent.date
print(newest_date)

2017-08-23


In [12]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database. 

# Calculate the date one year from the last date in data set.
from datetime import datetime
from dateutil.relativedelta import relativedelta
dt_new_date = datetime.strptime(newest_date, '%Y-%m-%d').date()

one_year_ago = dt_new_date - relativedelta(years = 1)

# Perform a query to retrieve the data and precipitation scores
from sqlalchemy import func
latest_year = session.query(Measurement.date,Measurement.prcp).\
filter(Measurement.date>=one_year_ago).all()

In [13]:
# Save the query results as a Pandas DataFrame and set the index to the date column
latest_df = pd.DataFrame.from_records(latest_year,
                                     index = 'Date',
                                     columns = ["Date",
                                               "Precipitation"])
# Sort the dataframe by date
latest_df.sort_index
latest_df.index = pd.to_datetime(latest_df.index)

In [14]:
latest_df.index.inferred_type == "string"

False

In [15]:
# Use Pandas Plotting with Matplotlib to plot the data

# latest_df.plot(kind = 'bar')
# plt.show()

In [16]:
# Use Pandas to calculate the summary statistics for the precipitation data


# Exploratory Station Analysis

In [17]:
# Design a query to calculate the total number stations in the dataset
no_of_stations = session.query(Measurement).group_by(Measurement.station).count()
print(f"There are {no_of_stations} stations used for this data set between {one_year_ago} and {newest_date}")

There are 9 stations used for this data set between 2016-08-23 and 2017-08-23


In [18]:
# Design a query to find the most active stations (i.e. what stations have the most rows?)
# List the stations and the counts in descending order.
station_counts = session.query(Measurement.station,func.count(Measurement.station))\
.group_by(Measurement.station).order_by(func.count(Measurement.station).desc())
for row in station_counts:
    print(row)


('USC00519281', 2772)
('USC00519397', 2724)
('USC00513117', 2709)
('USC00519523', 2669)
('USC00516128', 2612)
('USC00514830', 2202)
('USC00511918', 1979)
('USC00517948', 1372)
('USC00518838', 511)


In [21]:
stations_df = pd.DataFrame.from_records(station_counts,
                                        index = None,
                                       columns = ["Station", "Precipitation"])

stations_df.head()

TypeError: object of type 'Query' has no len()

In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.
most_active = session.query(Measurement.station)\
.group_by(Measurement.station).order_by(func.count(Measurement.station)\
.desc())
busiest_station = most_active[0]

In [ ]:
from sqlalchemy import and_
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram
busiest_station = session.query(Measurement.date,Measurement.tobs).\
filter(and_(Measurement.date>=one_year_ago,\
       Measurement.station.like(busiest_station))).all()
busy_df = pd.DataFrame.from_records(busiest_station,
                                     index = 'Date',
                                     columns = ["Date",
                                               "Temp Observation"])

# Close session

In [ ]:
# Close Session
session.close()